<a href="https://colab.research.google.com/github/DanielaSchacherer/IDC-Tutorials/blob/bmdeep_tutorial/notebooks/collections_demos/bonemarrowwsi_pediatricleukemia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BoneMarrowWSI-PediatricLeukemia


## Background

This notebook introduces the `BoneMarrowWSI-PediatricLeukemia` collection, which is presented in [this preprint](https://www.arxiv.org/pdf/2509.15895) and was recently added to [Imaging Data Commons](https://portal.imaging.datacommons.cancer.gov/).

- **Images**: The `BoneMarrowWSI-PediatricLeukemia` dataset comprises bone marrow aspirate smear WSIs for 246 pediatric cases of leukemia, including acute lymphoid leukemia (ALL), acute myeloid leukemia (AML), and chronic myeloid leukemia (CML). The smears were prepared for the initial diagnosis (i.e., without prior treatment), stained in accordance with the Pappenheim method, and scanned at 40x magnification.
- **Annotations**: The images have been annotated with rectangular regions of interest (ROI) of the evaluable monolayer area and more than 40000 cell bounding box annotations have been placed (with few exceptions) within the ROIs. For a subset of them all cells and other haematological structures have additionally been labelled by multiple experts in a consensus labeling approach with 49 distinct (cell type) classes. The consensus labelling approach worked as follows: each bounding box was successively labelled by different experts in so-called "annotation sessions" until (a) the bounding box had been labelled by at least two experts, and (b) the most frequent label had constituted at least half of all labels given to that bounding box (and had then been termed "consensus class"). In summary, the following annotations are available:  

    - For each slide: ROI annotations of the monolayer area for each slide
    - For some slides: Unlabeled cell bounding boxes
    - For some slides: Cell bounding boxes with cell type labels for each annotation session plus the finally obtained consensus.

This notebook concentrates on **how to access and work with the annotation data**, that are made available in **DICOM Microscopy Bulk Simple Annotation format (ANNs)**. As a general introduction to this format, we recommend having a look at [this tutorial notebook](https://github.com/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/pathomics/microscopy_dicom_ann_intro.ipynb).


<img src="https://raw.githubusercontent.com/ImagingDataCommons/IDC-Tutorials/master/notebooks/pathomics/bmdeep_annotations_example.png" alt="Example visualization of BoneMarrowWSI-PediatricLeukemia annotations" width="1000"/>



## Prerequisites
**Installations**
* **Install highdicom:** [highdicom](https://highdicom.readthedocs.io/en/latest/introduction.html) was specifically designed to work with DICOM objects holding image-derived information, e.g. annotations and measurements. Detailed information on highdicom's functionality can be found in its [user guide](https://highdicom.readthedocs.io/en/latest/usage.html).
* **Install wsidicom:** The [wsidicom](https://pypi.org/project/wsidicom/) Python package provides functionality to open and extract image or metadata from WSIs.
* **Install idc-index:** The Python package [idc-index](https://pypi.org/project/idc-index/) facilitates queries of the basic metadata and download of DICOM files hosted by the IDC.

In [1]:
%%capture
!pip install highdicom
!pip install wsidicom
!pip install idc-index --upgrade

## Imports

In [2]:
import os
import highdicom as hd
from idc_index import IDCClient
import pandas as pd
from google.cloud import storage
from pathlib import Path
from typing import List, Union

## Finding the `BoneMarrowWSI-PediatricLeukemia` dataset on IDC
To access and download image and ANNs files, we utilize the Python package [idc-index](https://github.com/ImagingDataCommons/idc-index) and fetch the `ann_index`, specific to DICOM ANN objects.

In [3]:
idc_client = IDCClient() # set-up idc_client
idc_client.fetch_index('ann_index') # fetch index for ANN objects

First, we verify that we have indeed 246 slides (=distinct SeriesInstanceUIDs of SM modality) in the `BoneMarrowWSI-PediatricLeukemia` collection:

In [5]:
query_slide_count = '''
SELECT COUNT(DISTINCT SeriesInstanceUID)
FROM
    index
WHERE
    collection_id = 'bonemarrowwsi_pediatricleukemia' AND Modality='SM'
'''
print(idc_client.sql_query(query_slide_count))

   count(DISTINCT SeriesInstanceUID)
0                                246


Next, we have a look at the available annotation (ANN) files. The following query collects information about ANN files on series-level from idc-index's `ann_index`.

In [ ]:
query_anns = '''
SELECT
    SeriesInstanceUID,
    SeriesDescription,
    StudyInstanceUID,
    referencedSeriesInstanceUID
FROM
    ann_index
WHERE
    collection_id = 'bonemarrowwsi_pediatricleukemia'
ORDER BY
    referencedSeriesInstanceUID
'''
annotations = idc_client.sql_query(query_anns)
display(annotations)

We can see, that for each slide (i.e. the **referencedSeriesInstanceUID**) there are multiple ANN Series. Looking at the **SeriesDescription**, we can assert what is described in the [Background](#Background) section of this notebook.

*   Each slide has "Monolayer regions of interest for cell classification" annotations.
*   For some slides, there is one ANN Series with "Unlabeled cell bounding boxes", while for others, there are multiple ANN Series containing "Cell bounding boxes with cell type labels" for different annotation sessions and the consensus labels.

We will use this knowledge of the **SeriesDescription** later in this notebook to facilitate filtering directly for labeled or unlabeled cell annotations.



## Viewing annotations


Annotations can be viewed and explored in detail on its respective slide using the Slim viewer. In the Slim viewer's interface at the bottom of the right sidebar you may select the ANN Series of interest to you from the drop-down menue, then click on `Annotation Groups` and switch the slider(s) to make annotations visible.

In [ ]:
viewer_url = idc_client.get_viewer_URL(studyInstanceUID=annotations['StudyInstanceUID'].iloc[3][0], viewer_selector='slim')
from IPython.display import IFrame
IFrame(viewer_url, width=1500, height=900)

## Accessing annotations

### Download complete annotation collection for local access
Since the complete set of annotation series is of reasonable size it could be downloaded completely using `idc_index` as shown below and then accessed from the local disk using `highdicom`.

In [ ]:
dcm_ann_dir = Path('/content/dicom_ann_annotations')
os.makedirs(dcm_ann_dir, exist_ok=True)

idc_client.download_from_selection(downloadDir=dcm_ann_dir,
                                   seriesInstanceUID=annotations['SeriesInstanceUID'].tolist(), dirTemplate=None)

For guidance on how to read the downloaded annotation files see section "Reading DICOM ANNs" of [this tutorial notebook](https://github.com/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/pathomics/microscopy_dicom_ann_intro.ipynb).

### Access annotations directly from the Cloud

A more desirable approach especially for larger size datasets is to directly extract the relevant information from the objects in the cloud. The following functions `get_roi_annotations()` and `get_cell_annotations()` can be used for this approach. They extract and summarize ROIs respectively cell annotations in an easy to use pandas DataFrame.
Note, that the selection of the respective annotation files, i.e. files containing ROI annotations, labeled or unlabeled cell annotations, is done by filtering for the respective SeriesDescription.

The following two code cells define and use `get_roi_annotations()` to select all DICOM ANNs in the `BoneMarrowWSI-PediatricLeukemia` collection that contain ROI annotations of the monolayer area.
The resulting pandas DataFrame contains
- **'SeriesInstanceUID'**: SeriesInstanceUID of the DICOM ANN Series containing the cell annotation.
- **'roi_id'**: the ID of the ROI
- **'roi_label'**: its label   
- **'roi_coordinates'**: the 2D coordinates in the image coordinate system of the referenced slide level
- **'reference_SeriesInstanceUID'** and **'reference_SOPInstanceUID'**: the SeriesInstanceUID and SOPInstanceUID of the slide level the annotations refer to. reference_SeriesInstanceUID can either be obtained from ann_index or read from the ANN file directly - for consistency with reference_SOPInstanceUID the later approach was chosen here.


In [ ]:
def get_roi_annotations(ann_to_process: int = 10):
    # We use the term 'monolayer' to filter for ANNs with ROIs in the SeriesDescription
    query_roi_anns = f'''
    SELECT
        SeriesInstanceUID
    FROM
        ann_index
    WHERE
        collection_id = 'bonemarrowwsi_pediatricleukemia'
        AND LOWER(SeriesDescription) LIKE '%monolayer%'
    ORDER BY
        SeriesInstanceUID
    LIMIT {ann_to_process}
    '''
    roi_series = idc_client.sql_query(query_roi_anns)
    rois = extract_rois(roi_series)
    return rois


def extract_rois(series_uids: List[str]) -> pd.DataFrame:
    gcs_client = storage.Client.create_anonymous_client()
    rows = []
    for series_uid in series_uids:
        file_urls = idc_client.get_series_file_URLs(seriesInstanceUID=series_uid, source_bucket_location='gcs')
        for file_url in file_urls:
            (_,_, bucket_name, folder_name, file_name) = file_url.split('/')
            bucket = gcs_client.bucket(bucket_name)
            blob = bucket.blob(f'{folder_name}/{file_name}')

            with blob.open('rb') as file_obj:
                ann = hd.ann.annread(file_obj)
                for ann_group in ann.get_annotation_groups():
                    coords = ann_group.get_graphic_data(coordinate_type='2D')
                    m_names, m_values, m_units = ann_group.get_measurements()
                    for c, m in zip(coords, m_values):
                        rows.append({
                            'SeriesInstanceUID': ann.SeriesInstanceUID,
                            'roi_id': int(m[0]), # allow empty roi_id,
                            'roi_label': ann_group.label,
                            'roi_coordinates': c,
                            'reference_SeriesInstanceUID': ann.ReferencedSeriesSequence[0].SeriesInstanceUID,
                            'reference_SOPInstanceUID': ann.ReferencedImageSequence[0].ReferencedSOPInstanceUID,
                        })
    rois = pd.DataFrame(rows)
    return rois

In [ ]:
# This code will run longer as you increase the number of ANN files to be processed
rois = get_roi_annotations(ann_to_process=10)
display(rois)

,SeriesInstanceUID,roi_id,roi_label,roi_coordinates,reference_SeriesInstanceUID,reference_SOPInstanceUID
0,1.2.826.0.1.3680043.10.511.3.76434139437749586...,2271,region_of_interest,"[[72032.0, 160247.0], [74080.0, 160247.0], [74...",1.2.826.0.1.3680043.8.498.98377665788926698337...,1.2.826.0.1.3680043.8.498.70616662305497812223...
1,1.2.826.0.1.3680043.10.511.3.76434139437749586...,2272,region_of_interest,"[[92857.0, 163260.0], [94905.0, 163260.0], [94...",1.2.826.0.1.3680043.8.498.98377665788926698337...,1.2.826.0.1.3680043.8.498.70616662305497812223...
2,1.2.826.0.1.3680043.10.511.3.76035111849294113...,1070,region_of_interest,"[[48995.0, 80570.0], [51043.0, 80570.0], [5104...",1.2.826.0.1.3680043.8.498.99045734331130228562...,1.2.826.0.1.3680043.8.498.52239720641745361153...
3,1.2.826.0.1.3680043.10.511.3.76035111849294113...,1071,region_of_interest,"[[99451.0, 126518.0], [101499.0, 126518.0], [1...",1.2.826.0.1.3680043.8.498.99045734331130228562...,1.2.826.0.1.3680043.8.498.52239720641745361153...
4,1.2.826.0.1.3680043.10.511.3.57387082213597634...,290,region_of_interest,"[[21001.0, 175192.0], [23049.0, 175192.0], [23...",1.2.826.0.1.3680043.8.498.36810224044030831386...,1.2.826.0.1.3680043.8.498.20301403784060697253...
5,1.2.826.0.1.3680043.10.511.3.57387082213597634...,291,region_of_interest,"[[85616.0, 154208.0], [87664.0, 154208.0], [87...",1.2.826.0.1.3680043.8.498.36810224044030831386...,1.2.826.0.1.3680043.8.498.20301403784060697253...
6,1.2.826.0.1.3680043.10.511.3.57387082213597634...,2323,region_of_interest,"[[27146.0, 122398.0], [47628.0, 122398.0], [47...",1.2.826.0.1.3680043.8.498.36810224044030831386...,1.2.826.0.1.3680043.8.498.20301403784060697253...
7,1.2.826.0.1.3680043.10.511.3.11224067190602751...,322,region_of_interest,"[[40807.0, 31720.0], [42855.0, 31720.0], [4285...",1.2.826.0.1.3680043.8.498.82223767803353692585...,1.2.826.0.1.3680043.8.498.72082594196695068782...
8,1.2.826.0.1.3680043.10.511.3.11224067190602751...,323,region_of_interest,"[[41251.0, 26908.0], [43299.0, 26908.0], [4329...",1.2.826.0.1.3680043.8.498.82223767803353692585...,1.2.826.0.1.3680043.8.498.72082594196695068782...
9,1.2.826.0.1.3680043.10.511.3.11224067190602751...,2017,region_of_interest,"[[25157.0, 41317.0], [30531.0, 41317.0], [3053...",1.2.826.0.1.3680043.8.498.82223767803353692585...,1.2.826.0.1.3680043.8.498.72082594196695068782...


The following code cells define and use `get_cell_annotations()` to select all DICOM ANNs in the `BoneMarrowWSI-PediatricLeukemia` collection that contain cell annotations. By setting the parameter 'subset' to either 'labeled', 'unlabeled' or 'both', it's possible to extract either only labeled, unlabeled or all cell annotations.
The resulting pandas DataFrame contains
- **'SeriesInstanceUID'**: SeriesInstanceUID of the DICOM ANN Series containing the cell annotation.
- **'annotation_session'**: 'n/a' for the unlabeled cells, otherwise the number of the annotation session or 'consensus' for the final consensus.
- **'cell_id'**: the ID of the cell
- **'roi_id'**: if applicable, the ID of the monolayer ROI, the cell is located within
- **'cell_label_code_scheme'**: Tuple of code of the cell label and designator of the coding scheme, e.g. (414387006, SCT) which is code 414387006 from SNOMED CT ontology
- **'cell_label'**: Code meaning of the cell label defined in cell_label_code_scheme e.g. 'Structure of haematological system'. Have a look at the [SNOMED Browser](https://browser.ihtsdotools.org/?perspective=full&conceptId1=414387006&edition=MAIN&release=&languages=en) for this example.
- **'cell_coordinates'**: the 2D coordinates in the image coordinate system of the referenced slide level
- **'reference_SeriesInstanceUID'** and **'reference_SOPInstanceUID'**: the SeriesInstanceUID and SOPInstanceUID of the slide level the annotations refer to. reference_SeriesInstanceUID can either be obtained from ann_index or read from the ANN file directly - for consistency with reference_SOPInstanceUID the later approach was chosen here.

In [ ]:
def get_cell_annotations(subset: Union[str, None] = None, ann_to_process: int = 10) -> pd.DataFrame:
    assert subset in ['labeled', 'unlabeled', None]
    query_words = {'labeled' : 'labels', 'unlabeled': 'unlabeled'}
    if subset == 'labeled' or subset == 'unlabeled':
        # Use the respective query word to filter on ANNs with labeled or unlabeled cells in the SeriesDescription
        query_word = query_words[subset]
        query_cell_anns = f'''
          SELECT
              SeriesInstanceUID,
          FROM
              ann_index
          WHERE
              collection_id = 'bonemarrowwsi_pediatricleukemia'
              AND LOWER(SeriesDescription) LIKE '%{query_word}%'
          ORDER BY
              SeriesInstanceUID
          LIMIT {ann_to_process}
          '''
    else:
      query_cell_anns = f'''
          SELECT
              SeriesInstanceUID,
          FROM
              ann_index
          WHERE
              collection_id = 'bonemarrowwsi_pediatricleukemia'
          ORDER BY
              SeriesInstanceUID
          LIMIT {ann_to_process}
          '''
    cell_series = idc_client.sql_query(query_cell_anns)
    cells = extract_cells(cell_series)
    return cells


def extract_cells(series_uids: List[str]) -> pd.DataFrame:
    gcs_client = storage.Client.create_anonymous_client()
    rows = []
    for series_uid in series_uids:
        file_urls = idc_client.get_series_file_URLs(seriesInstanceUID=series_uid, source_bucket_location='gcs')
        for file_url in file_urls:
            (_,_, bucket_name, folder_name, file_name) = file_url.split('/')
            bucket = gcs_client.bucket(bucket_name)
            blob = bucket.blob(f'{folder_name}/{file_name}')

            with blob.open('rb') as file_obj:
                ann = hd.ann.annread(file_obj)
                for ann_group in ann.get_annotation_groups():
                    coords = ann_group.get_graphic_data(coordinate_type='2D')
                    m_names, m_values, m_units = ann_group.get_measurements()
                    for c, m in zip(coords, m_values):
                        rows.append({
                            'SeriesInstanceUID': ann.SeriesInstanceUID,
                            'annotation_session': get_annotation_session(ann),
                            'cell_id': int(m[0]),
                            'roi_id': int(m[1]) if m.size > 1 else None, # allow empty roi_id,
                            'cell_label': ann_group.annotated_property_type.meaning,
                            'cell_label_code_scheme': (ann_group.annotated_property_type.value, ann_group.annotated_property_type.scheme_designator),
                            'cell_coordinates': c,
                            'reference_SeriesInstanceUID': ann.ReferencedSeriesSequence[0].SeriesInstanceUID,
                            'reference_SOPInstanceUID': ann.ReferencedImageSequence[0].ReferencedSOPInstanceUID
                        })
    cells = pd.DataFrame(rows)
    return cells


def get_annotation_session(ann: hd.ann.sop.MicroscopyBulkSimpleAnnotations) -> str:
    if 'unlabeled' in ann.SeriesDescription.lower():
        return 'n/a'
    return ann.SeriesDescription.split(':')[0]

In [ ]:
# This code will run longer as you increase the number of ANN files to be processed
unlabeled_cells = get_cell_annotations(subset='unlabeled', ann_to_process=10)
display(unlabeled_cells)

,SeriesInstanceUID,annotation_session,cell_id,roi_id,cell_label,cell_label_code_scheme,cell_coordinates,reference_SeriesInstanceUID,reference_SOPInstanceUID
0,1.2.826.0.1.3680043.10.511.3.51699668688633439...,n/a,104693,1070,Structure of haematological system,"(414387006, SCT)","[[49259.0, 80647.0], [49424.0, 80647.0], [4942...",1.2.826.0.1.3680043.8.498.99045734331130228562...,1.2.826.0.1.3680043.8.498.52239720641745361153...
1,1.2.826.0.1.3680043.10.511.3.51699668688633439...,n/a,104694,1070,Structure of haematological system,"(414387006, SCT)","[[49383.0, 80751.0], [49514.0, 80751.0], [4951...",1.2.826.0.1.3680043.8.498.99045734331130228562...,1.2.826.0.1.3680043.8.498.52239720641745361153...
2,1.2.826.0.1.3680043.10.511.3.51699668688633439...,n/a,104695,1070,Structure of haematological system,"(414387006, SCT)","[[49178.0, 80526.0], [49270.0, 80526.0], [4927...",1.2.826.0.1.3680043.8.498.99045734331130228562...,1.2.826.0.1.3680043.8.498.52239720641745361153...
3,1.2.826.0.1.3680043.10.511.3.51699668688633439...,n/a,104696,1070,Structure of haematological system,"(414387006, SCT)","[[49262.0, 80516.0], [49358.0, 80516.0], [4935...",1.2.826.0.1.3680043.8.498.99045734331130228562...,1.2.826.0.1.3680043.8.498.52239720641745361153...
4,1.2.826.0.1.3680043.10.511.3.51699668688633439...,n/a,104697,1070,Structure of haematological system,"(414387006, SCT)","[[49723.0, 80551.0], [49856.0, 80551.0], [4985...",1.2.826.0.1.3680043.8.498.99045734331130228562...,1.2.826.0.1.3680043.8.498.52239720641745361153...
...,...,...,...,...,...,...,...,...,...
1655,1.2.826.0.1.3680043.10.511.3.36659336595099377...,n/a,109220,1125,Structure of haematological system,"(414387006, SCT)","[[82138.0, 330089.0], [82200.0, 330089.0], [82...",1.2.826.0.1.3680043.8.498.69631392888544866375...,1.2.826.0.1.3680043.8.498.33316914385579021210...
1656,1.2.826.0.1.3680043.10.511.3.36659336595099377...,n/a,109221,1125,Structure of haematological system,"(414387006, SCT)","[[82988.0, 329818.0], [83057.0, 329818.0], [83...",1.2.826.0.1.3680043.8.498.69631392888544866375...,1.2.826.0.1.3680043.8.498.33316914385579021210...
1657,1.2.826.0.1.3680043.10.511.3.36659336595099377...,n/a,109222,1125,Structure of haematological system,"(414387006, SCT)","[[82997.0, 329744.0], [83113.0, 329744.0], [83...",1.2.826.0.1.3680043.8.498.69631392888544866375...,1.2.826.0.1.3680043.8.498.33316914385579021210...
1658,1.2.826.0.1.3680043.10.511.3.36659336595099377...,n/a,109223,1125,Structure of haematological system,"(414387006, SCT)","[[82103.0, 329810.0], [82205.0, 329810.0], [82...",1.2.826.0.1.3680043.8.498.69631392888544866375...,1.2.826.0.1.3680043.8.498.33316914385579021210...


In [ ]:
# This code will run longer as you increase the number of ANN files to be processed
labeled_cells = get_cell_annotations(subset='labeled', ann_to_process=10)
sorted_cell_labels = labeled_cells.sort_values(by=['reference_SOPInstanceUID', 'cell_id', 'annotation_session'])
display(sorted_cell_labels.style.hide(axis='index')) # don't show row index

SeriesInstanceUID,annotation_session,cell_id,roi_id,cell_label,cell_label_code_scheme,cell_coordinates,reference_SeriesInstanceUID,reference_SOPInstanceUID
1.2.826.0.1.3680043.10.511.3.18476424701131582063353881362630884,Consensus,40235,290.000000,Smudge cell,"('34717007', 'SCT')",[[ 20941. 175124.] [ 21072. 175124.] [ 21072. 175266.] [ 20941. 175266.]],1.2.826.0.1.3680043.8.498.36810224044030831386861436640243108164,1.2.826.0.1.3680043.8.498.20301403784060697253200659813926783376
1.2.826.0.1.3680043.10.511.3.56631485211069664746302234685573715,Session 0,40235,290.000000,Smudge cell,"('34717007', 'SCT')",[[ 20941. 175124.] [ 21072. 175124.] [ 21072. 175266.] [ 20941. 175266.]],1.2.826.0.1.3680043.8.498.36810224044030831386861436640243108164,1.2.826.0.1.3680043.8.498.20301403784060697253200659813926783376
1.2.826.0.1.3680043.10.511.3.59892159514053659950705832114602849,Session 1,40235,290.000000,Smudge cell,"('34717007', 'SCT')",[[ 20941. 175124.] [ 21072. 175124.] [ 21072. 175266.] [ 20941. 175266.]],1.2.826.0.1.3680043.8.498.36810224044030831386861436640243108164,1.2.826.0.1.3680043.8.498.20301403784060697253200659813926783376
1.2.826.0.1.3680043.10.511.3.18476424701131582063353881362630884,Consensus,40236,290.000000,Structure of haematological system,"('414387006', 'SCT')",[[ 21012. 175152.] [ 21161. 175152.] [ 21161. 175286.] [ 21012. 175286.]],1.2.826.0.1.3680043.8.498.36810224044030831386861436640243108164,1.2.826.0.1.3680043.8.498.20301403784060697253200659813926783376
1.2.826.0.1.3680043.10.511.3.56631485211069664746302234685573715,Session 0,40236,290.000000,Structure of haematological system,"('414387006', 'SCT')",[[ 21012. 175152.] [ 21161. 175152.] [ 21161. 175286.] [ 21012. 175286.]],1.2.826.0.1.3680043.8.498.36810224044030831386861436640243108164,1.2.826.0.1.3680043.8.498.20301403784060697253200659813926783376
1.2.826.0.1.3680043.10.511.3.59892159514053659950705832114602849,Session 1,40236,290.000000,Structure of haematological system,"('414387006', 'SCT')",[[ 21012. 175152.] [ 21161. 175152.] [ 21161. 175286.] [ 21012. 175286.]],1.2.826.0.1.3680043.8.498.36810224044030831386861436640243108164,1.2.826.0.1.3680043.8.498.20301403784060697253200659813926783376
1.2.826.0.1.3680043.10.511.3.51986836579885900118376658443649996,Session 2,40236,290.000000,Structure of haematological system,"('414387006', 'SCT')",[[ 21012. 175152.] [ 21161. 175152.] [ 21161. 175286.] [ 21012. 175286.]],1.2.826.0.1.3680043.8.498.36810224044030831386861436640243108164,1.2.826.0.1.3680043.8.498.20301403784060697253200659813926783376
1.2.826.0.1.3680043.10.511.3.43936597653709648655229500365098661,Session 3,40236,290.000000,Structure of haematological system,"('414387006', 'SCT')",[[ 21012. 175152.] [ 21161. 175152.] [ 21161. 175286.] [ 21012. 175286.]],1.2.826.0.1.3680043.8.498.36810224044030831386861436640243108164,1.2.826.0.1.3680043.8.498.20301403784060697253200659813926783376
1.2.826.0.1.3680043.10.511.3.84929085837168107515323645734404637,Session 4,40236,290.000000,Structure of haematological system,"('414387006', 'SCT')",[[ 21012. 175152.] [ 21161. 175152.] [ 21161. 175286.] [ 21012. 175286.]],1.2.826.0.1.3680043.8.498.36810224044030831386861436640243108164,1.2.826.0.1.3680043.8.498.20301403784060697253200659813926783376
1.2.826.0.1.3680043.10.511.3.18476424701131582063353881362630884,Consensus,40237,290.000000,Lymphoblast,"('15433008', 'SCT')",[[ 20955. 175275.] [ 21065. 175275.] [ 21065. 175379.] [ 20955. 175379.]],1.2.826.0.1.3680043.8.498.36810224044030831386861436640243108164,1.2.826.0.1.3680043.8.498.20301403784060697253200659813926783376


## How to use the `BoneMarrowWSI-PediatricLeukemia` annotations
The `BoneMarrowWSI-PediatricLeukemia` collection stands out due to the extensive amount of information contained in its annotations. More than 40000 cells are annotated with bounding boxes suitable for training **cell detection models**, 28000 of those additionally received expert-generated class labels for **cell type classification** tasks. Particularly noteworthy is the uncertainty information embedded in the consensus labelling process, giving insight into which cell types are particularly challenging to determine or easy to confuse with others.
In the cell below, we catch some of those cases:

In [ ]:
grouped_cell_labels = sorted_cell_labels.groupby('cell_id').agg({'cell_label': list, 'cell_label_code_scheme': list,
                                                              'reference_SOPInstanceUID': 'first',
                                                              'cell_coordinates': 'first'})
uncertain = grouped_cell_labels['cell_label'].apply(lambda x: len(set(x)) > 1)
display(grouped_cell_labels[uncertain])

,cell_label,cell_label_code_scheme,reference_SOPInstanceUID,cell_coordinates
cell_id,,,,
47204,"[Structure of haematological system, Unusable ...","[(414387006, SCT), (111235, DCM), (414387006, ...",1.2.826.0.1.3680043.8.498.72082594196695068782...,"[[42780.0, 32381.0], [42827.0, 32381.0], [4282..."


# Next steps

Share your feedback or ask questions about this notebook in IDC Forum: https://discourse.canceridc.dev.

If you are interested in tissue type annotations or want to learn about DICOM Structured Reporting, you can take a look at [this notebook](https://github.com/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/collections_demos/rms_mutation_prediction/RMS-Mutation-Prediction-Expert-Annotations_exploration.ipynb) navigating expert-generated region annotations for rhabdomyosarcoma tumor slides.